# Getting Started

In [1]:
import os
import numpy as np
import pandas as pd
import datetime
import re

import numpy as np
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import math
from matplotlib import rc
import random

from collections import Counter, defaultdict
import matplotlib.ticker as ticker
from math import sqrt

import plotly.express as px 
import matplotlib.pyplot as plt
import warnings

In [2]:
datetime.datetime.now()

datetime.datetime(2022, 6, 16, 10, 49, 41, 69001)

In [3]:
cols = ["song", "songID", "artist", "album", "genre", "play_count", "click_count"]
test_df = pd.read_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/test_dataset_small_1.csv')
    
test_df
    
    

,song,songID,artist,album,genre,play_count,click_count
0,Soul Food,1,Mat Zo,Self Assemble,Electronic,82294,555
1,The Enemy,2,Mat Zo,Self Assemble,Electronic,42875,777
2,Everytime We Touch,3,Cascada,Everytime We Touch,Electronic,1756387293,9872342342
3,Something in the Way,4,Nirvana,Nevermind,Grunge,171857763,223746531
4,heat,5,53 Thieves,after hours,Electronic,88342,101527
5,Another Life,6,Jadu Heart,Hyper Romance,Electronic,17009,28032
6,Numb,7,Elderbrook,Why Do We Shake In The Cold?,Electronic,441070,565088
7,Control,8,Emmit Fenn,The Last Dance,Electronic,267650,310823
8,Fire for You,9,Cannons,Shadows,Electronic,32199,37825
9,Dope,10,Demeter,Dope,Electronic,1780336,2138853


---
# Global Objects and Classes

Nodes, Networks, etc

In [4]:
class Node:

    nodeID = None
    library = None
    clicklog = None
    
    def __init__(self, newID):
        self.nodeID = newID
        self.library = pd.DataFrame()
        self.clicklog = pd.DataFrame()
        
    
    def updateLibrary(self, newData):
        self.library = self.library.append(newData, ignore_index=True)
        
        
    def gossip_library(self, neighbors, to_gossip, debug):
        
        timestamp = datetime.datetime.now()
        for n in neighbors:
#             message = Message(this.nodeID, n.nodeID, to_gossip, timestamp)
            temp = n.library
            n.library = pd.concat([n.library, to_gossip]).drop_duplicates()
            n.library.reset_index(inplace=True, drop=True)
            if(debug): 
                print("Added", to_gossip)
                print(" to", temp)
                print("creating: ", n.library)
            temp = None
        

In [5]:
class Message:
    
    fromID = None
    toID = None
    contents = None
    timestamp = None
    
    
    def __init__(self, fromID, toID, contents, timestamp):
        self.fromID = fromID
        self.toID = toID
        self.contents = contents

In [6]:
# class Network:
    
#     nodes = None
#     default_cols = None    
#     library = None
    
    
#     def __init__(self):
#         self.default_cols = pd.Series()
#         self.library = pd.DataFrame()
#         self.nodes = pd.Series()
    

#     def __init__(self, numNodes, default_cols, default_df):
#         self.default_cols = default_cols
#         self.library = default_df
#         self.nodes = []
        
#         for i in range(numNodes):
            
#             newNode = Node(i)
#             newNode.library = self.library
#             self.nodes.append(newNode)

    
#     def gossip(self, num_nodes):
        


In [7]:
def buildNetwork(numNodes, default_df):
    
    network = []
    
    for i in range(numNodes):
        
        newNode = Node(i)
        newNode.library = default_df
        network.append(newNode)
        
    return network

In [8]:
def buildNetwork_v2(numNodes, default_df):
    
    network = []
    
    for i in range(numNodes):
        
        newNode = Node(i)
        newNode.library = default_df.sample(frac=np.random.rand(), axis=0, ignore_index=True)
        network.append(newNode)
        
    return network

---
# Major Algorithms

- Search
    - (v1) Must be able to return all string matches
    - Ideally ranks based on popularity (clicklog)
    - No user profiling/machine learning (yet)

- Gossip
    - Pass messages occasionally (when/why?)
    - 


- Rank

### Search


In this iteration, I will assume that the first node that it finds the result in is also the most efficient to reach, therefore I will only return the location of the first result (instead of a list of all containing nodes).

Problem with this is: what if each node has a different dataset? We can't end the search at just the first hit. We need to go through everything nearby. **Therefore I think it's better to keep a list of all contained nodes, and limit the search in some other way to constrain it**.


In [9]:
# debug = False

# def search(term, to_search):
    
#     # We need to aggregate the results where we have a match
#     total_results = pd.DataFrame()
#     contained_nodes = []
#     first_found = -1
    
#     # Look through the list of nodes to search through
#     for node in to_search:
#         if(debug): print("node:", node.nodeID)
        
#         # For each column in the library of each node
#         for col in node.library.columns:
                
#             # Get the columns of where the term is found in the node library
#             node_results = node.library.iloc[np.where(node.library[col].astype(str).str.find(term) == 0)]
            
#             if(debug): 
#                 print("column:", col)
#                 print("node_results:", node_results)
            
#             # Concatenate the total results into a dataframe
#             total_results = pd.concat([node_results, total_results]).drop_duplicates()
            
#             # Keep track of the nodes that the result is found in
#             if(len(node_results) > 0):
#                 print("node", node.nodeID, "contained the search term:", term)
#                 total_results["found_in"] = node.nodeID
                
       
#     return total_results

### Search/rank v1:

- Search all nodes
- Return all literal string matches
    - including a list of every node it was found on
- Sort the matches based purely on click count

In [10]:
####
# This iteration collects all nodes that the result is found in
####


debug = False

def search(term, to_search):
    
    term = term.lower()
    
    # We need to aggregate the results where we have a match
    total_results = pd.DataFrame()
    contained_nodes = []
    
    
    # Look through the list of nodes to search through
    for node in to_search:
        if(debug): print("node:", node.nodeID)
        
        # For each column in the library of each node
        for col in node.library.columns:
                
            # Get the columns of where the term is found in the node library
            node_results = node.library.iloc[np.where(node.library[col].astype(str).str.contains(term, na=False, case=False))]
            
            if(debug): 
                print("column:", col)
                print("node_results:", node_results)
            
            # Concatenate the total results into a dataframe
            total_results = pd.concat([node_results, total_results]).drop_duplicates()
            
            # Keep track of the nodes that the result is found in
            if(len(node_results) > 0):
                if(debug): print("node", node.nodeID, "contained the search term:", term)
                contained_nodes.append(node.nodeID)
       
    
    # add the list of contained nodes to the returned dataset
#     for i in range(len(total_results)):
#         total_results['contained_nodes'] = []
#         total_results.assign(contained_in = [contained_nodes])     

    if(debug): print("contained nodes:", contained_nodes)
        
    total_results = total_results.reset_index(drop=True, inplace=False)

    return ranked_results(total_results), contained_nodes


def ranked_results(results):
    
    return results.sort_values(by='click_count', ascending=False)

### Search and Rank

This goes beyond just ranking based on click. How?

- ?
  

In [11]:
def searchAndRank(term, toSearch):
    
    
    
    return rank_v1(search_results), contained_nodes

### Gossip

In [20]:
numNodes = 25
network = buildNetwork_v2(numNodes, test_df)

In [21]:
network[0].library

,song,songID,artist,album,genre,play_count,click_count
0,Something in the Way,4,Nirvana,Nevermind,Grunge,171857763,223746531
1,heat,5,53 Thieves,after hours,Electronic,88342,101527


In [22]:
max_n = 3

for node in network:
    
    neighbors = random.sample(network, int(np.random.rand() * max_n))
    node.gossip_library(neighbors, node.library, True)

Added                    song  songID      artist        album       genre  \
0  Something in the Way       4     Nirvana    Nevermind      Grunge   
1                  heat       5  53 Thieves  after hours  Electronic   

   play_count  click_count  
0   171857763    223746531  
1       88342       101527  
 to                  song  songID      artist                         album  \
0                Numb       7  Elderbrook  Why Do We Shake In The Cold?   
1                Dope      10     Demeter                          Dope   
2  Everytime We Touch       3     Cascada            Everytime We Touch   

        genre  play_count  click_count  
0  Electronic      441070       565088  
1  Electronic     1780336      2138853  
2  Electronic  1756387293   9872342342  
creating:                     song  songID      artist                         album  \
0                  Numb       7  Elderbrook  Why Do We Shake In The Cold?   
1                  Dope      10     Demeter             

In [23]:
network[7].library

,song,songID,artist,album,genre,play_count,click_count
0,Control,8,Emmit Fenn,The Last Dance,Electronic,267650,310823
1,Dope,10,Demeter,Dope,Electronic,1780336,2138853
2,Something in the Way,4,Nirvana,Nevermind,Grunge,171857763,223746531
3,The Enemy,2,Mat Zo,Self Assemble,Electronic,42875,777
4,Everytime We Touch,3,Cascada,Everytime We Touch,Electronic,1756387293,9872342342
5,Fire for You,9,Cannons,Shadows,Electronic,32199,37825
6,Soul Food,1,Mat Zo,Self Assemble,Electronic,82294,555
7,Another Life,6,Jadu Heart,Hyper Romance,Electronic,17009,28032
8,heat,5,53 Thieves,after hours,Electronic,88342,101527
9,Numb,7,Elderbrook,Why Do We Shake In The Cold?,Electronic,441070,565088


---
# Testing Environment

In [24]:
numNodes = 10

# build network where each node has the full dataset
# network = buildNetwork(numNodes, test_df)


# build network where each node has 25% of the full dataset
network = buildNetwork_v2(numNodes, test_df)

In [25]:
search("Electronic", network)

(                 song  songID      artist                         album  \
 6  Everytime We Touch       3     Cascada            Everytime We Touch   
 0                Dope      10     Demeter                          Dope   
 5                Numb       7  Elderbrook  Why Do We Shake In The Cold?   
 4             Control       8  Emmit Fenn                The Last Dance   
 7                heat       5  53 Thieves                   after hours   
 8        Fire for You       9     Cannons                       Shadows   
 3        Another Life       6  Jadu Heart                 Hyper Romance   
 1           The Enemy       2      Mat Zo                 Self Assemble   
 2           Soul Food       1      Mat Zo                 Self Assemble   
 
         genre  play_count  click_count  
 6  Electronic  1756387293   9872342342  
 0  Electronic     1780336      2138853  
 5  Electronic      441070       565088  
 4  Electronic      267650       310823  
 7  Electronic       88342  

In [26]:
# I need to find a way to ignore all cases. Make everything caps?
search("the", network)[0]

,song,songID,artist,album,genre,play_count,click_count
4,Something in the Way,4,Nirvana,Nevermind,Grunge,171857763,223746531
2,Numb,7,Elderbrook,Why Do We Shake In The Cold?,Electronic,441070,565088
1,Control,8,Emmit Fenn,The Last Dance,Electronic,267650,310823
3,Another Life,6,Jadu Heart,Hyper Romance,Electronic,17009,28032
0,The Enemy,2,Mat Zo,Self Assemble,Electronic,42875,777


In [27]:
test_df.iloc[np.where(node.library.song.astype(str).str.contains("you", na=False, case=False))]

,song,songID,artist,album,genre,play_count,click_count
6,Numb,7,Elderbrook,Why Do We Shake In The Cold?,Electronic,441070,565088


---
# Methods and code to save for later

In [28]:
# Lambda expression to see if a term is in a column of a dataframe
test_df.apply(lambda row: row.astype(str).str.contains('Mat Zo').any(), axis=1)

0     True
1     True
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool

In [213]:
# Regex to ignore case in string search, can only compare two string objects though (afaik)
re.search("soul", test_df.iloc[0].song, re.IGNORECASE)

<re.Match object; span=(0, 4), match='Soul'>